# Алгоритмы ГОСТ34-2018

## [ГОСТ34.12-2018](https://files.stroyinf.ru/Data/705/70509.pdf) (Кузнечик)

Симметричный алгоритм блочного шифрования, состоит из 10 раундов. Оперирует блоками размером **128 бит**, мастер ключом размером **256 бит** и раундовыми ключами размером **128 бит**. В основе алгоритма лежит использование SP-сетей (подстановочно-перестановочные сети).
Такой шифр получает на вход блок, ключ и совершает несколько раундов (в алгоритме Кузнечик полных раундов - 9), состоящих из стадий подстановки и перестановки.

Раунды алгоритма Кузнечик содержат 3 последовательные операции:
1)	**Побитовый XOR ключа и блока данных (X)**. Перед началом каждого раунда очередной 128-битовой входной вектор складывается с раундовым ключом.
2)	**Нелинейное преобразование (S)**: замена одного байта входного вектора на другой в соответствии с фиксированной подстановкой.
3)	**Линейное преобразование (L)**: над каждым байтом блока производится операция умножения, определенная в поле Гаула по модулю неприводимого многочлена степени 8, на один из коэффициентов ряда в зависимости от порядкового номера байта. Байты складываются между собой по модулю 2, и все 16 байт блока сдвигаются в сторону младшего разряда. 
**Последний 10-й раунд не полный. Он включает в себя только операцию XOR с ключом.**

Формирование раундовых ключей происходит из мастер-ключа. Первая пара ключей является результатом деления мастер-ключа пополам. Последующие получаются посредством применения 8 итераций сетей Фейстеля совместно с некоторой константой, которая получается из применения линейного преобразования к значению номера итерации.
![jupiter](./fejstel.png)

В результате, шифрование одного 128-битного входного блока a описывается следующим уравнением: 
$E(a)=X[K_{10}]LSX[K_{9}]…LSX[K_{1}](a)$
Расшифрование реализуется обращением преобразований и применением их в обратном порядке: 
$D(a)=X[K_{1}] S^{-1} L^{-1} X[K_{2}]…S^{-1} L^{-1} X[K_{9}] S^{-1} L^{-1} X[K_{10}](a)$

### Примеры использования

In [191]:
from gost34.grasshopper import grasshopper
import binascii

Ввиду того, что во всех положениях ГОСТ34 описана работа с текстом, представленным в 16-ричном формате, для большей наглядности напишем вспомогательную функцию, которая будет переводить текстовые сообщения в hex формат и дополнять их незначащими нулями до размера, кратного 4n бит:

In [192]:
def str_to_hex(message, n):
    hex_message = message.encode('CP866').hex() # переводим сообщение в hex формат
    
    if len(hex_message) == n:
        return hex_message
    else:
        # дополняем нулями до размера, кратного 4n бит
        padded_hex_message = hex_message.ljust(len(hex_message) + (n - len(hex_message) % n), "0")
    
    return padded_hex_message

И также функция, которая bytearray переводит сначала в hex, а затем в привычный текст:

In [193]:
def hex_to_str(message):
    hex_decrypt = binascii.hexlify(bytearray(message)).decode('CP866')
    text_message = bytes.fromhex(hex_decrypt).decode('CP866')
    
    return text_message

### Шифрование

In [194]:
message_1 = str_to_hex("qqqqqqqqqqqqqqqq", 32) #исходное сообщение
key_1 = binascii.unhexlify('8899aabbccddeeff0011223344556677fedcba98765432100123456789abcdef') #мастер-ключ (в hex формате)

gost3412_1 = grasshopper(key_1) #создаем экземпляр класса на мастер-ключе
message_encrypt_1 = gost3412_1.encryption(binascii.unhexlify(message_1)) #шифруем сообщение

print(hex_to_str(message_encrypt_1)) #зашифрованное сообщение в текстовом формате
binascii.hexlify(bytearray(message_encrypt_1)).decode('CP866') #зашифрованное сообщение в hex формате

▒Цe #ЎО┌┼O
┐Ж┤


'b19665ff23f68edac57f4f150abf86b4'

Шифрование длинного сообщения

In [195]:
message_2 = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum sollicitudin fermentum urna, quis tempor lectus posuere id. Aenean vitae ipsum efficitur, auctor urna sagittis, pretium sapien'
            
block_size = 16 #размер блока алгоритма в символах
blocks = [message_2[i:i+block_size] for i in range(0, len(message_2), block_size)] #разбиваем сообщение на блоки длины блоков базового алгоритма

key_2 = binascii.unhexlify('8899aabbccddeeff0011223344556677fedcba98765432100123456789abcdef') #мастер-ключ
gost3412_2 = grasshopper(key_2) #экземпляр класса на мастер-ключе

message_encrypt_2 = [] #итоговый массив, в который будет записывать зашифрованниые блоки

for block in blocks:
    block = str_to_hex(block, 32)
    message_encrypt_2.extend(gost3412_2.encryption(binascii.unhexlify(block))) #каждый блок шифруем и записываем его в итоговый массив

print(hex_to_str(message_encrypt_2)) #зашифрованное сообщение в текстовом формате
binascii.hexlify(bytearray(message_encrypt_2)).decode('CP866') #зашифрованное сообщение в hex формате

C|╜аw|ЄR╦Уk╙:/·^k8-


'f740881ca2d535902825a8a143d07d803a4c9b95757c5dd450d1c0304ba1e8034190fc56874d4bef237dafb8e4ff43c32c46c0796f1d54802eadecb0dc53d9cbcff894bf3c390c31d58b401fa20f151e22fa58f0d0dd7e2a97e361f2746b68fcb8954ccbfbbdc912a0b7d3a9f62563dbe6dbafd59a82ba07582b5e6dab8cadd1f5761efacc1a951ab5976920605058a8cec05ec93eeeef326247c0d8dc7d1b52b2a43f227287bcdb7555868e3f89f1929e76767091f11b74210d437cbda077117cf20352cb7f936bd33a2ffa5e6b382d'

### Дешифрование

In [196]:
key_1 = binascii.unhexlify('8899aabbccddeeff0011223344556677fedcba98765432100123456789abcdef') 
gost3412_1 = grasshopper(key_1) #создаем экземпляр класса на известном мастер-ключе

message_decrypt_1 = gost3412_1.decryption(message_encrypt_1) #дешифруем сообщение

print(hex_to_str(message_decrypt_1))

qqqqqqqqqqqqqqqq


In [197]:
encrypted_blocks = [message_encrypt_2[i:i+block_size] for i in range(0, len(message_encrypt_2), block_size)] #создаем массив зашифрованных блоков длины базового алгоритма

key_2 = binascii.unhexlify('8899aabbccddeeff0011223344556677fedcba98765432100123456789abcdef')
gost3412_2 = grasshopper(key_2) #создаем экземпляр класса на известном мастер-ключе

message_decrypt_2 = [] #создаем итоговый массив, в который будем записывать дешифрованные блоки

for block in encrypted_blocks:
    message_decrypt_2.extend(gost3412_2.decryption(block)) #каждый блок дешифруем и записываем в итоговый массив


print(hex_to_str(message_decrypt_2))

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum sollicitudin fermentum urna, quis tempor lectus posuere id. Aenean vitae ipsum efficitur, auctor urna sagittis, pretium sapien              


## [ГОСТ34.12-2018](https://files.stroyinf.ru/Data/705/70509.pdf) (Магма)

Алгоритм шифрования, который также описан в стандарте ГОСТ 34.12-2018. Использует **64-битовые** блоки, **256-битовый** мастер-ключ и **32-битные** раундовые ключи. Алгоритм проходит **32 раунда** и использует следующие преобразования:

Первоначально входной блок разбивается на 2 вектора по 32 бит: $a_{0}$ и $a_{1}$. 
Функция преобразования $G[k](a_{1}, a_{0})$ принимает упорядоченную пару этих векторов и возвращает упорядоченную пару такого вида: $G[k](a_{1}, a_{0})=(a_{0}, g[k](a_{0})⨁a_{1})$
**То есть элементы меняются местами и второй элемент возвращается с применением XOR к результату функции g на раундовом ключе k и значении первого элемента.**

Функция g является **линейным преобразованием** и работает следующим образом:
$g[k](a)=(t(Vec_{32} (Int_{32} (a)⊞Int_{32} (k)))) ⋘_{11}$
Образы вектора a и раундового ключа k в кольце $Z_{2^{32}}$ складываются и отображаются обратно в 32-битовый вектор. Далее, к этому результату применяется нелинейная функция постановки t, и наконец, происходит 11-битовый циклический сдвиг над результатом работы функции t.
Функция t разбивает входной 32-битовый вектор на 8 4-битных вектора и применяет константную подстановку π к каждому из них: 
$t(a)=t(a_{7}∥⋯∥a_{0})=π_{7} (a_{7})∥⋯∥π_{0} (a_{0})$

32-й раунд отличается от остальных: функция преобразования $G^{*}$ принимает упорядоченную пару, но возвращает уже единственное значение – 64-битный выходной вектор, **являющийся искомым шифром**:
$G^* [k](a_{1}, a_{0})=(g[k](a_{0})⊕a_{1})∥a_{0}$

**Развертывание мастер-ключа K** происходит следующим образом: первые 8 раундовых ключей происходит простым делением мастер-ключа на 32-битные последовательности. Соответственно первый раундовый ключ – это старшие 32 бит мастер-ключа, восьмой – младшие. Следующие 8 ключей, как и следующие вплоть до 24-го аналогичны первым восьми. То есть 9 и 17 ключи совпадают с первым, 16 и 24 совпадают с восьмым. Последние 8 ключей получаются обратным образом: 25-ый ключ аналогичен восьмому, а 32-ой – первому. Итого 32 итерационных ключа.

**Общая формула алгоритма**: $E(a)=G^* [K_{32}]G[K_{31}]…G[K_{2}]G[K_{1}](a_{1},a_{0})$
Расшифрование происходит в зависимости от итерационных ключей и реализует обратную подстановку: $D(a)=G^* [K_{1}]G[K_{2}]…G[K_{31}]G[K_{32}](a_{1}, a_{2})$

Значения константных подстановок (из ГОСТ 34.12-2018):  
$π_{0}=(12,4,6,2,10,5,11,9,14,8,13,7,0,3,15,1)$  
$π_{1}=(6,8,2,3,9,10,5,12,1,14,4,7,1,13,0,15)$  
$π_{2}=(11,3,5,8,2,15,10,13,14,1,7,4,12,9,6,0)$  
$π_{3}=(12,8,2,1,13,4,15,6,7,0,10,5,3,14,9,11)$  
$π_{4}=(7,15,5,10,8,1,6,13,0,9,3,14,11,4,2,12)$  
$π_{5}=(5,13,15,6,9,2,12,10,11,7,8,1,4,3,14,0)$  
$π_{6}=(8,14,2,5,6,9,1,12,15,4,11,0,13,10,3,7)$  
$π_{7}=(1,7,14,13,0,5,8,3,4,15,10,6,9,12,11,2)$

### Примеры использования

In [198]:
from gost34.magma import magma

### Шифрование

In [199]:
message_1 = str_to_hex('abcdwfgq', 16)

key_1 = binascii.unhexlify('8899aabbccddeeff0011223344556677fedcba98765432100123456789abcdef')
gost3412_1 = magma(key_1)

message_encrypt_1 = gost3412_1.encrypt(list(binascii.unhexlify(message_1)))

print(hex_to_str(message_encrypt_1))
binascii.hexlify(bytearray(message_encrypt_1)).decode('CP866')

х_c5D:╡`


'e55f6335443ab560'

In [200]:
message_2 = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum sollicitudin fermentum urna, quis tempor lectus posuere id. Aenean vitae ipsum efficitur, auctor urna sagittis, pretium sapien'

block_size = 8 #размер блока алгоритма
blocks = [message_2[i:i+block_size] for i in range(0, len(message_2), block_size)]

key_2 = binascii.unhexlify('ffeeddccbbaa99887766554433221100f0f1f2f3f4f5f6f7f8f9fafbfcfdfeff')
gost3412_2 = magma(key_2)

message_encrypt_2 = [] #итоговый массив, в который будет записывать зашифрованниые блоки

for block in blocks:
    block = str_to_hex(block, 16)
    message_encrypt_2.extend(gost3412_2.encrypt(list(binascii.unhexlify(block))))


print(hex_to_str(message_encrypt_2))
binascii.hexlify(bytearray(message_encrypt_2)).decode('CP866')

╟(ппЕрr▐╩)д░▀[8eХ▒1│)h╟MrJ!fУзв▒3¤ь═оaT─A┴┘ЩУ╨╤и▌W%6с┼Э
P▓ЬUe╕╖1Ъ╩(╛M9o╬ЎыТ│жрєLhSюи╝Т"`╫tB/*╡{Ч╘╨ё¤/3ит<їп:я▓Az·╤ИYLц%ы1ЖМЩP▒Ю[S0Ю


'c728afaf1f85e072deca29a4b01f1319df5b386595b131b32968c74d18724a211d66930ca703a2b11933fdec19cdae19611354c44105c1d9991a061493d0d1a8dd572536e1c59d0a7661e264b800f70d82ec28ccb821f4942343fd5b5726fa4d7a9d1f84512e9a056bb20d50b29c5565b8b71e01319aca28be4d160f391c6fcef6eb92b3a6e0f34c6853eea8bc922260d77406421f2f05112ab57b97d414d0f1fd2f33a8e21c3cf50516af3aefb21a01417afad10b88594ce625eb3186088c9950b19e135b53309e'

### Дешифрование

In [201]:
key_1 = binascii.unhexlify('8899aabbccddeeff0011223344556677fedcba98765432100123456789abcdef')
gost3412_1 = magma(key_1)

message_decrypt_1 = gost3412_1.decrypt(message_encrypt_1)

print(hex_to_str(message_decrypt_1))

abcdwfgq


In [202]:
encrypted_blocks = [message_encrypt_2[i:i+block_size] for i in range(0, len(message_encrypt_2), block_size)]

key_2 = binascii.unhexlify('ffeeddccbbaa99887766554433221100f0f1f2f3f4f5f6f7f8f9fafbfcfdfeff')
gost3412_2 = magma(key_2)

message_decrypt_2 = [] #создаем итоговый массив, в который будем записывать дешифрованные блоки

for block in encrypted_blocks:
    message_decrypt_2.extend(gost3412_2.decrypt(block)) #каждый блок дешифруем и записываем в итоговый массив


print(hex_to_str(message_decrypt_2))

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum sollicitudin fermentum urna, quis tempor lectus posuere id. Aenean vitae ipsum efficitur, auctor urna sagittis, pretium sapien      


## [ГОСТ34.11-2018](https://files.stroyinf.ru/Data/705/70506.pdf) (Стрибог)

**Стрибог512** – хеш-функция с длиной выходного значения **512 бит**. Входные данные – блок длиной **512 бит**. Если длина сообщения больше 512 бит – происходит усечение сообщения. Последний блок дополняется до длины 512 бит. Функции преобразования алгоритма:

1) **XOR (X)**: $X[k](a)=k⨁a$, k и a – 512-битовые векторы
2) **Нелинейное преобразование, функция подстановки (S)**: 
   $S(a)=S(a_{63}∥⋯∥a_{0})= π(a_{63})∥⋯∥π(a_{0})$ – каждый байт входного 512-битового вектора a заменяется соответствующим        байтом из      таблицы подстановки π.
3) **Линейное преобразование, функция перестановки и умножения (P и L)**:
   $P(a)=P(a_{63}∥⋯∥a_{0})=a_{τ(63)}∥⋯∥a_{τ(0)}$  – каждая пара байтов входной 512-битовой последовательности происходит замена одного    байта другим в соответствии с таблицей перестановок τ.
   $L(a)=L(a_{7}∥⋯∥a_{0})=l(a_{7})∥⋯∥l(a_{0})$ – входной 512-битовый вектор разбивается на 8 64-битных вектора, к каждому из которых      применяется умножение справа на матрицу A над полем Гаула 2-го порядка.

**Основная функция алгоритма** – функция сжатия $g_{N}$, которая принимает на вход упорядоченную пару 512-битовых векторов и возвращает один 512-битовый вектор. Она использует вспомогательную 13-раундовую функцию E двух переменных, которая описывается следующим образом:
$E(K,m)=X[K_{13}]LPSX[K_{12}]…LPSX[K_{2}]LPSX[K_{1}](m)$, где K-некоторый 512-битовый мастер-ключ, $K_{i}$ – раундовые ключи. **Раундовые ключи вычисляются следующим образом**: первый раундовый ключ совпадает с мастер-ключом, дальнейшие получаются подстановкой-перестановкой результата побитового сложения предыдущего раундового ключа и раундовой константы:
$K_{i}=LPS(K_{i-1}⊕C_{i-1})$
Первые 12 раундов функции E – это XOR подстановка и перестановка (LPSX) раундового ключа и некоторых входных данных m. Последний раунд – обычный XOR.

**Функция сжатия описывается следующим образом**:  
$g_{N} (h,m)=E(LPS(h⊕N),m)⊕h⊕m$ – входные значения побитово складываются с результатом выполнения функции E над подстановкой-перестановкой (LPS) XOR 1-го входного значения с некоторым значением N (этот результат используется как мастер-ключ) и вторым входным значением m. 
**Сам алгоритм реализуется в 3 этапа.**

**1 этап (присваивание начальных значений)**: переменной h присваивается значение вектора инициализации, который в свою очередь равен $0^{512}$. Переменным N и Σ также присваивается значение $0^{512}$.

**2 этап (усечение входной последовательности данных M)**: если |M|≥512, то определяется значение m, равное последним 512 битам M. Далее переопределяются h,N и Σ:

$h=g_{N} (h,m)$  
$N=Vec_{512} (Int_{512} (N)⊞512)$  
$Σ=Vec_{512} (Int_{512} (Σ)⊞Int_{512} (m))$  
После этого из M, удаляются последние 512 бит. Алгоритм этого этапа применяется до тех пор, пока М будет не больше 511 бит.  

**3 этап (дополнение входной последовательности и вычисление хеш-кода)**:
$m=0^{511-|M|} ∥1∥M$ – определяем/переопределяем m  
Переопределяем h,N и Σ:  
$h=g_{N} (h,m)$  
$N=Vec_{512} (Int_{512} (N)⊞|M|)$  
$Σ=Vec_{512} (Int_{512} (Σ)⊞Int_{512} (m))$  
Еще 2 раза переопределяется h:  
$h=g_{0} (h,N)$  
$h=g_{0} (h,Σ)$  
**Конечное значение h является искомым.**  
  
Таблица подстановок π и таблица подстановок τ совпадают с таблицами в алгоритме Кузнечик.

**Стрибог256** – хеш-функция аналогичная Стрибог512, за исключением некоторых моментов:  
1) Выходное значение – 256-битовый хеш-код.  
2) Вектор инициализации равен $(00000001)^{64}$  
3) На 3 этапе, добавляется еще одно, третье переопределение переменной h:  
$h=MSB_{256} (h)$ – от 512-битового h отсекаются младшие 256 бит.

### Примеры использования
### Стрибог256

In [203]:
from gost34.streebog256 import streebog256

In [204]:
message = str_to_hex('Hello World!', 128)

gost3411_256 = streebog256(binascii.unhexlify(message))

digest = gost3411_256.digest()

print(hex_to_str(digest))
binascii.hexlify(digest).decode('CP866')

▀Аъ


'886232f90c9836d9648ea41c4f8e132b89077598f9d4602275f5c6df0ddf80ea'

In [205]:
message = str_to_hex('wygfiewidnwrwfgcsifsbgsvdkfygnksydvbkzsgvkzsdbkufgbzdsykgzsduygvusdgbyhdkbvgfnysugfvbusgyduy', 128)

gost3411_256 = streebog256(binascii.unhexlify(message))

digest = gost3411_256.digest()

print(hex_to_str(digest))
binascii.hexlify(digest).decode('CP866')

ЄF┌Q=╨;Т№{М2¤_▓╕а╒  ъа┴t┬&в


'19f246da513dd00c3b92fc7b8c32fd5f1610b2b8a0d52020eaa0c11474c226a2'

### Стрибог512

In [206]:
from gost34.streebog512 import streebog512

In [207]:
message = str_to_hex('crypto', 128)

gost3411_512 = streebog512(binascii.unhexlify(message))

digest = gost3411_512.digest()

print(hex_to_str(digest))
binascii.hexlify(digest).decode('CP866')

ь3Ob\5╤о┘▒<ъZ┬>К╧(─8╔щ╕<8УУЭ)zDk═ PИYs5╘3"/и:╤u9?їs*>


'ec334f625c35d1aed905b13cea5ac23e8acf2804c40b38c9e9b83c3893939d29197a446b01cd00507f8859147335d4013308222fa83ad1751739063ff5732a3e'

In [208]:
message = str_to_hex('aaaioioioiioiqjsdneijfweifwiefnieowejfiwnfiehfniwenfijwenf', 128)

gost3411_512 = streebog512(binascii.unhexlify(message))

digest = gost3411_512.digest()

print(hex_to_str(digest))
binascii.hexlify(digest).decode('CP866')

0░/Е╬ьаЯ?Y├╦o┘д/[┼∙zд░▌x.■>еЧ┴С╨╣╕qў8ы\w▌ш1Kqly■хnБc]│


'30b02f181f0785ceeca09f143f59c316cb6fd9a4052f5bc508f97aa415b0dd782efe3ea597c191d0b9b871f738eb5c1a77dd1fe8314b716c79fee56e81635db3'

## [ГОСТ34.10-2018](https://files.stroyinf.ru/Data/705/70512.pdf) (Алгоритмы формирования и проверки ЭЦП)

**Формирование подписи**  
1) Первоначально из входного сообщения M вычисляется двоичный вектор хеш-кода h длины l бит по алгоритму Стрибог.  
2) Далее, вычисляется целое число \begin{equation}α=\sum_{i=0}^{l-1} α_{i} 2^{i}\end{equation}где $α_{i}$ – i-ый бит полученного хеш-кода. Значение α берется по модулю q, где q – порядок подгруппы группы точек эллиптической кривой, и приравнивается к переменной e (если e оказалось 0, то переопределяем его как 1).  
3) Затем генерируется псевдослучайное число k: 0 < k < q и вычисляется точка эллиптической кривой C = kP (P – точка кривой порядка q, т.е. qP = 0). После этого определяется переменная r: $r=x_{c}$ (mod q),где $x_{c}$-x координата точки C. Если r = 0, то повторяем данный пункт заново.  
4) Далее вычисляется значение s: s = (rd + ke)(mod q), где d – ключ подписи. Если  s = 0, необходимо вернуться к пункту 3.  
5) В результате конкатенации двоичных векторов r || s получаем искомую цифровую подпись ζ.
  
Использование хеш-функции зависит от эллиптической кривой и порядка q. 
Если $2^{254}<q<2^{256}$ то используется Стрибог256, если $2^{508}<q<2^{512}$ то используется Стрибог512.

**Проверка подписи**

1) По подписи ζ вычисляем целые числа r и s. Если 0 < r < q и 0 < s < q – переходим к следующему пункту. Иначе – подпись неверна.
2) Вычисляется хеш-код сообщения M и значения α и e аналогично п.2) в разделе «Формирование подписи».
3) Вычисляется $v=e^{-1}$ (mod q) и по нему находятся
   $z_{1}=sv(mod q)$, $z_{2}=-rv(mod q)$
4) Вычисляется точка эллиптической кривой $С=z_{1} P+z_{2} Q$, где Q – ключ проверки подписи – точка эллиптической кривой Q: dP = Q. Далее определяется значение $R=x_{c} (mod q)$
5) Если R = r – подпись принимается, иначе – подпись неверна.

In [209]:
from gost34.gost3410 import DSGOST

### Примеры использования
### Задаем параметры эллиптической кривой:

In [210]:
p = 57896044618658097711785492504343953926634992332820282019728792003956564821041 #модуль 
a = 7
b = 43308876546767276905765904595650931995942111794451039583252968842033849580414 #коэффициенты
x = 2
y = 4018974056539037503335449422937059775635739389905545080690979365213431566280 #координаты
q = 57896044618658097711785492504343953927082934583725450622380973592137631069619 #порядок

private_key = 55441196065363246126355624130324183196576709222340016572108097750006097525544

### Подпись

In [211]:
message_hash = int(binascii.hexlify(digest).decode('Windows-1251'), 16)

gost3410 = DSGOST(p, a, b, q, x, y)
sign = gost3410.sign(message_hash, private_key)
sign

(48040265884241376626627835808967751096720262965568645644336884349912716925063,
 26429302642830877647902491075943663743358728645895341078457801108525276118070)